<a href="https://colab.research.google.com/github/MithilaCoomaraswamy/DSGP/blob/Exercise-Recommender/Exercise_Rec_Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
# Load the exercise dataset
data = pd.read_csv('Exercise dataset.csv')

In [3]:
# Encode 'BMIcase' into numerical values
label_encoder = LabelEncoder()
data['BMIcase_encoded'] = label_encoder.fit_transform(data['BMIcase'])

In [4]:
# Define features (X) and target (y)
X = data[['Weight', 'Height', 'BMI', 'Age', 'BMIcase_encoded']]
y = data['Exercise Intensity']

In [5]:
# Standardize the features for Random Forest
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [6]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [12]:
# Initialize and train the Random Forest model
random_forest = RandomForestClassifier(
    random_state=42,
    n_jobs=-1,
    max_depth=10,                 # Limiting the depth of each tree
    min_samples_split=10,         # Minimum samples to split an internal node
    min_samples_leaf=5,           # Minimum samples in a leaf node
    max_features='sqrt',          # Number of features to consider for the best split
    n_estimators=100
    )
random_forest.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, min_samples_leaf=5, min_samples_split=10,
                       n_jobs=-1, random_state=42)

In [13]:
# Making the predictions on the test set
y_pred = random_forest.predict(X_test)

# Training accuracy
train_accuracy = random_forest.score(X_train, y_train)
print("Training Accuracy: ", train_accuracy)

# Test accuracy
test_accuracy = random_forest.score(X_test, y_test)
print("Test Accuracy: ", test_accuracy)

# Print evaluation metrics
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Training Accuracy:  1.0
Test Accuracy:  1.0
Classification Report:
               precision    recall  f1-score   support

           1       1.00      1.00      1.00         4
           2       1.00      1.00      1.00         5
           3       1.00      1.00      1.00        15
           4       1.00      1.00      1.00       114
           5       1.00      1.00      1.00       112
           6       1.00      1.00      1.00       101
           7       1.00      1.00      1.00       148

    accuracy                           1.00       499
   macro avg       1.00      1.00      1.00       499
weighted avg       1.00      1.00      1.00       499

Confusion Matrix:
 [[  4   0   0   0   0   0   0]
 [  0   5   0   0   0   0   0]
 [  0   0  15   0   0   0   0]
 [  0   0   0 114   0   0   0]
 [  0   0   0   0 112   0   0]
 [  0   0   0   0   0 101   0]
 [  0   0   0   0   0   0 148]]


In [17]:
# Function to get user recommendations
def get_user_recommendations_rf(weight, height, age):
    # Calculate BMI
    bmi = weight / (height ** 2)

    # Derive BMI case based on BMI value
    if bmi < 16:
        bmi_case = 'sever thinness'
    elif 16 <= bmi < 17:
        bmi_case = 'moderate thinness'
    elif 17 <= bmi < 18.5:
        bmi_case = 'mild thinness'
    elif 18.5 <= bmi < 24.9:
        bmi_case = 'normal'
    elif 25 <= bmi < 29.9:
        bmi_case = 'over weight'
    elif 30 <= bmi < 34.9:
        bmi_case = 'obese'
    else:
        bmi_case = 'severe obese'

    # Encode the derived BMI case
    bmi_case_encoded = label_encoder.transform([bmi_case])[0]

    # Standardize input features
    input_features = scaler.transform([[weight, height, bmi, age, bmi_case_encoded]])

    # Predict Exercise Intensity
    predicted_intensity = random_forest.predict(input_features)[0]

    # Map intensity to exercises
    gym_exercise = data.loc[data['Exercise Intensity'] == predicted_intensity, 'Gym Exercise'].iloc[0]
    home_exercise = data.loc[data['Exercise Intensity'] == predicted_intensity, 'Home Exercise'].iloc[0]

    return predicted_intensity, gym_exercise, home_exercise, bmi_case

# Example user input
predicted_intensity, gym_exercise, home_exercise, bmi_case = get_user_recommendations_rf(
    weight=75, height=1.7, age=44
)
print(f"Predicted Exercise Intensity: {predicted_intensity}\n")
print(f"Recommended Gym Exercise: {gym_exercise}\n")
print(f"Recommended Home Exercise: {home_exercise}\n")
print(f"BMI Case: {bmi_case}\n")

Predicted Exercise Intensity: 5

Recommended Gym Exercise: Treadmill Walking with Intervals; Moderate Weightlifting (Upper and Lower Body); Stationary Biking (High Resistance Intervals); Resistance Circuit Training

Recommended Home Exercise: Modified Burpees; Step-Ups on a Sturdy Chair or Step; Squat Jumps; Glute Bridge Variations

BMI Case: over weight



/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
